In [ ]:
import pandas as pd
import os
import sys
sys.path.append(os.path.abspath(".."))
from utils.data_utils import create_dimension_table, create_bridge_table, parse_flexible_date, create_date_dimension


### Data Normalization 

The goal of this notebook is to document the normalisation process of the data. Please scroll down to the bottom for a summary of the actions taken to this end.

In [9]:
data = pd.read_csv("../data/shows_cleaned.csv")

In [10]:
df = data.copy()

In [11]:
df["genres"].head(20)

0                                           Documentary
1                                           Documentary
2                                           Documentary
3                               Documentary,Crime,Drama
4                                   Documentary,History
5                          Crime,Drama,Mystery,Thriller
6                                          Comedy,Drama
7                                                Comedy
8             Documentary,Biography,Crime,History,Sport
9                                           Crime,Drama
10    Animation,Action,Adventure,Drama,Fantasy,Sci-F...
11                                   Comedy,Drama,Music
12                                    Documentary,Music
13                                        Drama,Romance
14                            Drama,Fantasy,History,War
15                                         Comedy,Drama
16                                 Comedy,Drama,Romance
17                                          Docu

In [12]:
print(f"\nContains comma: {df['genres'].str.contains(',', na=False).sum()}")
print(f"Contains semicolon: {df['genres'].str.contains(';', na=False).sum()}")

print(f"\nGenres NaNs: {df['genres'].isna().sum()}")


Contains comma: 2506
Contains semicolon: 0

Genres NaNs: 0


In [14]:
all_genres = df['genres'].dropna().str.split(',').explode().str.strip()
print(f"\nTotal unique genres: {all_genres.nunique()}")
print(f"\nUnique genres:\n{sorted(all_genres.unique())}")


Total unique genres: 26

Unique genres:
['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'Unknown', 'War', 'Western']


In [15]:
dim_genres = create_dimension_table(df, 'genres')
print(f"Rows: {len(dim_genres)}")
print(dim_genres)

Rows: 26
    genre_id   genre_name
0          1       Action
1          2    Adventure
2          3    Animation
3          4    Biography
4          5       Comedy
5          6        Crime
6          7  Documentary
7          8        Drama
8          9       Family
9         10      Fantasy
10        11    Game-Show
11        12      History
12        13       Horror
13        14        Music
14        15      Musical
15        16      Mystery
16        17         News
17        18   Reality-TV
18        19      Romance
19        20       Sci-Fi
20        21        Sport
21        22    Talk-Show
22        23     Thriller
23        24      Unknown
24        25          War
25        26      Western


In [16]:
print(f"\nContains comma: {df['top_cast'].str.contains(',', na=False).sum()}")
print(f"Contains semicolon: {df['top_cast'].str.contains(';', na=False).sum()}")

# Check for NaNs
print(f"\nDirector NaNs: {df['top_cast'].isna().sum()}")


Contains comma: 3284
Contains semicolon: 0

Director NaNs: 0


In [18]:
all_writer = df['top_cast'].dropna().str.split(',').explode().str.strip()
print(f"\nTotal unique actors: {all_writer.nunique()}")



Total unique actors: 34890


In [19]:
print(f"\nContains comma: {df['writer'].str.contains(',', na=False).sum()}")
print(f"Contains semicolon: {df['writer'].str.contains(';', na=False).sum()}")

# Check for NaNs
print(f"\nDirector NaNs: {df['writer'].isna().sum()}")

all_actors = df['writer'].dropna().str.split(',').explode().str.strip()
print(f"\nTotal unique writer: {all_actors.nunique()}")
#print(f"\nUnique writer:\n{sorted(all_actors.unique()[0:5])}")


Contains comma: 2641
Contains semicolon: 0

Director NaNs: 0

Total unique writer: 7932


In [11]:
dim_genres = create_dimension_table(df, 'genres')
dim_writers = create_dimension_table(df, 'writer')
dim_directors = create_dimension_table(df, 'director')
dim_cast = create_dimension_table(df, 'top_cast')

dim_creators = create_dimension_table(df, 'created_by')
dim_creators = dim_creators.rename(columns={"created_by_id": "creator_id","created_by_name": "creator_name"})

dim_production_companies = create_dimension_table(df, 'production_companies')
dim_production_companies = dim_production_companies.rename(columns={"production_companie_id": "production_company_id","production_companie_name": "production_company_name"})


In [ ]:
# Bridge tables
bridge_genres = create_bridge_table(df, 'genres', dim_genres)
bridge_cast = create_bridge_table(df, 'top_cast', dim_cast)
bridge_writers = create_bridge_table(df, 'writer', dim_writers)
bridge_directors = create_bridge_table(df, 'director', dim_directors)
bridge_companies = create_bridge_table(df, 'production_companies', dim_production_companies, id_prefix='production_company')
bridge_creators = create_bridge_table(df,"created_by",dim_creators,id_prefix="creator")

### Release date column

In [13]:
print(df['releaseDate'].dtype)

object


In [14]:

df['releaseDate'].astype(str).str.len().value_counts().sort_index()

releaseDate
4      191
7       47
10    3065
11      11
Name: count, dtype: int64

In [15]:
df['date_length'] = df['releaseDate'].astype(str).str.len()

df.sort_values('date_length')[['title', 'releaseDate', 'date_length']]

,title,releaseDate,date_length
3267,Sex Box,2015,4
3252,So You Think You Can Dance,2005,4
3263,The Return of Jezebel James,2008,4
3260,The Last Templar,2009,4
3186,9JKL,2017,4
...,...,...,...
1623,Muhammad Ali's Greatest Fight,05 Oct 2013,11
2825,Party Girl,09 Jun 1995,11
2472,41,21 Jul 2015,11
1776,Barbershop,13 Sep 2002,11


In [ ]:
df['releaseDate'] = df['releaseDate'].apply(parse_flexible_date)
print(f"NaT count: {df['releaseDate'].isna().sum()}")
print(f"Date range: {df['releaseDate'].min()} to {df['releaseDate'].max()}")



NaT count: 0
Date range: 1969-07-21 00:00:00 to 2025-12-27 00:00:00


In [17]:

df.drop(columns=['date_length'], inplace=True)

In [ ]:
dim_date = create_date_dimension('1969-01-01', '2025-12-31')
dim_date.to_csv('power_bi_data/dim_date.csv', index=False)
print(f"Rows: {len(dim_date)}")

Rows: 20819


In [ ]:
df_fact = df.drop(columns=['genres', 'top_cast', 'writer', 'director', 'production_companies','created_by'])
df_fact = df_fact.rename(columns={"id": "show_id"})

In [104]:
df_fact.head(3)

,id,title,releaseDate,seasonCount,rating,description,duration,tagline,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,seasonCount_was_null
0,1000358361,Planet Earth: Blue Planet II,2017-10-29,1.0,TV-G,"Airing simultaneously on AMC, BBC America, IFC...",50.0,Take a deep breath,97.0,7,Universal acclaim,82,178,Universal acclaim,False
1,1000359012,America to Me,2018-08-26,1.0,TV-14,The 10-part documentary series from Steve Jame...,60.0,NaN,96.0,9,Universal acclaim,59,75,Mixed or average,False
2,1000357720,Planet Earth II,2016-11-06,1.0,TV-G,"Narrated by David Attenborough, the sequel to ...",50.0,A new world revealed,96.0,10,Universal acclaim,92,242,Universal acclaim,False


In [ ]:


# Export all tables
df_fact.to_csv('power_bi_data/fact_shows.csv', index=False)

# Dimensions
dim_genres.to_csv('power_bi_data/dim_genres.csv', index=False)
dim_cast.to_csv('power_bi_data/dim_cast.csv', index=False)
dim_writers.to_csv('power_bi_data/dim_writers.csv', index=False)
dim_directors.to_csv('power_bi_data/dim_directors.csv', index=False)
dim_production_companies.to_csv('power_bi_data/dim_companies.csv', index=False)
dim_creators.to_csv('power_bi_data/dim_creators.csv', index=False)
dim_date.to_csv('power_bi_data/dim_date.csv', index=False)

# Bridges
bridge_genres.to_csv('power_bi_data/bridge_genres.csv', index=False)
bridge_cast.to_csv('power_bi_data/bridge_cast.csv', index=False)
bridge_writers.to_csv('power_bi_data/bridge_writers.csv', index=False)
bridge_directors.to_csv('power_bi_data/bridge_directors.csv', index=False)
bridge_companies.to_csv('power_bi_data/bridge_companies.csv', index=False)
bridge_creators.to_csv('power_bi_data/bridge_creators.csv', index=False)

print("Exported 12 tables to power_bi_data/:")
print(f"  fact_shows.csv: {len(df_fact)} rows")
print(f"  dim_genres.csv: {len(dim_genres)} rows")
print(f"  dim_cast.csv: {len(dim_cast)} rows")
print(f"  dim_writers.csv: {len(dim_writers)} rows")
print(f"  dim_directors.csv: {len(dim_directors)} rows")
print(f"  dim_companies.csv: {len(dim_production_companies)} rows")
print(f"  dim_date.csv: {len(dim_date)} rows")
print(f"  bridge_genres.csv: {len(bridge_genres)} rows")
print(f"  bridge_cast.csv: {len(bridge_cast)} rows")
print(f"  bridge_writers.csv: {len(bridge_writers)} rows")
print(f"  bridge_directors.csv: {len(bridge_directors)} rows")
print(f"  bridge_companies.csv: {len(bridge_companies)} rows")

Exported 12 tables to power_bi_data/:
  fact_shows.csv: 3314 rows
  dim_genres.csv: 26 rows
  dim_cast.csv: 34890 rows
  dim_writers.csv: 7932 rows
  dim_directors.csv: 2387 rows
  dim_companies.csv: 6873 rows
  dim_date.csv: 684 rows
  bridge_genres.csv: 9044 rows
  bridge_cast.csv: 64043 rows
  bridge_writers.csv: 12296 rows
  bridge_directors.csv: 5395 rows
  bridge_companies.csv: 44088 rows


### Data Normalization Summary - TV Shows Dataset

### Overview

Normalized a flat TV shows dataset into a star schema for Power BI analysis.

---

### Final Schema

#### Fact Table

| Table | Description | Key |
|-------|-------------|-----|
| fact_shows | Main TV show data with measures | show_id (PK) |

**Columns retained:**
- id, title, releaseDate, seasonCount, duration
- metascore, userscore, userscore_count
- rating, userscore_sentiment, metascore_sentiment
- tagline, description
- seasonCount_was_null (flag)

---

#### Dimension Tables

| Table | Columns | Rows |
|-------|---------|------|
| dim_genres | genre_id, genre_name | 26 |
| dim_cast | cast_id, cast_name |34890 |
| dim_writers | writer_id, writer_name | 7932 |
| dim_directors | director_id, director_name | 2387 |
| dim_companies | company_id, company_name | 6873   |
| dim_creators | creator_id, creator_name | 1935 |
| dim_date | date, year, month, month_name, year_month | ~20,800 |

---

#### Bridge Tables (Many-to-Many)

| Table | Columns |
|-------|---------|
| bridge_genres | show_id, genre_id |
| bridge_cast | show_id, cast_id |
| bridge_writers | show_id, writer_id |
| bridge_directors | show_id, director_id |
| bridge_companies | show_id, company_id |

---

### Reusable Functions Created

#### create_dimension_table()

```python
def create_dimension_table(df, column_name, id_prefix=None, delimiter=','):
    """
    Create a dimension table from a multi-value column.
    Returns DataFrame with {prefix}_id and {prefix}_name columns.
    """
```

#### create_bridge_table()

```python
def create_bridge_table(df, column_name, dim_df, show_id_col='id', id_prefix=None, delimiter=','):
    """
    Create a bridge table linking shows to a dimension.
    Returns DataFrame with show_id and {prefix}_id columns.
    """
```

#### create_date_dimension()

```python
def create_date_dimension(start_date, end_date):
    """
    Create a daily date dimension table.
    Returns DataFrame with date, year, month, month_name, year_month.
    """
```

---

### Date Handling

#### Issue
releaseDate had 4 different formats:
- Length 4: `2004` (year only)
- Length 7: `2004-06` (year-month)
- Length 10: `2004-06-05` (full date)
- Length 11: `04 Aug 2014` (OMDB format)

#### Solution

```python
def parse_flexible_date(date_str):
    """
    Parse dates in various formats, defaulting to first of month/year.
    """
```

- Year only → `YYYY-01-01`
- Year-month → `YYYY-MM-01`
- Full date → preserved
- OMDB format → parsed with `%d %b %Y`

#### Date Range
1969-07-21 to 2025-12-27

---

### Power BI Relationships

```
fact_shows.id ──────┬──> bridge_genres.show_id
                    ├──> bridge_cast.show_id
                    ├──> bridge_writers.show_id
                    ├──> bridge_directors.show_id
                    └──> bridge_companies.show_id

bridge_genres.genre_id ────> dim_genres.genre_id
bridge_cast.cast_id ───────> dim_cast.cast_id
bridge_writers.writer_id ──> dim_writers.writer_id
bridge_directors.director_id -> dim_directors.director_id
bridge_companies.company_id -> dim_companies.company_id

fact_shows.releaseDate ────> dim_date.date
```

---


### Export Location

All files exported to `power_bi_data/` folder:
- fact_shows.csv
- dim_genres.csv
- dim_cast.csv
- dim_writers.csv
- dim_directors.csv
- dim_companies.csv
- dim_date.csv
- bridge_genres.csv
- bridge_cast.csv
- bridge_writers.csv
- bridge_directors.csv
- bridge_companies.csv